# Fairness Post-Processing on the Adult Income Dataset

In this notebook we compare several post-processing techniques on the well-known "Adult" dataset to  
reduce demographic disparities in a binary income prediction task. 

We’ll walk through:  

1. Imports & configuration

2. Utility functions (data loading, feature selection)

3. Training / post‑processing loop

4. Result summary & feature importance

5. Conclusions

## 1  Imports & basic setup

In [ ]:
import sys
import os
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset, AdultDataset

# TODO: change the import method
repo_root = os.path.dirname(os.getcwd())
sys.path.insert(0, repo_root)
repair_folder = os.path.join(repo_root, "humancompatible", "repair")
sys.path.insert(0, repair_folder)
from humancompatible.repair.methods.cost import *
from humancompatible.repair.methods.coupling_utils import *
from humancompatible.repair.methods.data_analysis import *
from humancompatible.repair.group_blind_repair import *
from humancompatible.repair.methods.metrics import *

from humancompatible.repair.postprocess.roc_postprocess import ROCpostprocess
from humancompatible.repair.postprocess.proj_postprocess import Projpostprocess

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

We’ve turned off FutureWarning noise so we can focus on the core outputs. Next up, we’ll write a couple of utility functions to:

1. **Load & clean** the Adult dataset (apply binning, encode labels).

2. **Compute TV distances** to pick out the most imbalanced features.

## 2  Utility helpers

Below are three helper functions:

- **`load_data`**: merges train/test, binning continuous features, encodes sensitive (`S`) & target (`Y`).

- **`categorise`**: assigns numeric bins for age, hours-per-week, capital gain/loss.

- **`choose_x`**: measures total-variation distance for each feature to detect imbalance and returns a shortlist for repair.

In [3]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

In [4]:
data_path='..//data//adult'
var_list=['hours-per-week','age','capital-gain','capital-loss','education-num'] #
pa='race'
favorable_label = 1
var_dim=len(var_list)

K=200
e=0.01

if pa == 'sex':
    thresh=0.05
elif pa == 'race':
    thresh=0.05

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]

In [5]:
tv_dist

{'hours-per-week': np.float64(0.12216173195089294),
 'age': np.float64(0.04149230147335384),
 'capital-gain': np.float64(0.026764553949230142),
 'capital-loss': np.float64(0.014217783478743178),
 'education-num': np.float64(0.11867963282506956)}

We can see which features differ most across the sensitive groups:

- **hours-per-week** and **education-num** stand out  

- These will be the coordinates along which we apply our post-processing repairs

With `X, y` ready, we now move on to the main experiment loop.

## 3  Training & post‑processing experiment

Here we’ll:

1. Split data (train/val/test).  

2. Fit a Random Forest baseline.  

3. Apply four post-processing strategies:
   - **origin** (no fairness correction)  
   - **unconstrained** (repair without thresholding)  
   - **barycentre** (optimal transport-based repair)  
   - **partial** (partial coupling)  
   - **ROC** (learn an ROC-based thresholding on the validation set)

We repeat this 10× to smooth out randomness.

In [6]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.01,0.1),theta=1e-2)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv('../data/report_postprocess_adult_'+str(pa)+'.csv',index=None)

Optimal classification threshold (with fairness constraints) = 0.2500
Optimal ROC margin = 0.0278
Optimal classification threshold (with fairness constraints) = 0.2500
Optimal ROC margin = 0.0278
Optimal classification threshold (with fairness constraints) = 0.2100
Optimal ROC margin = 0.0233
Optimal classification threshold (with fairness constraints) = 0.2100
Optimal ROC margin = 0.0233
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.2500
Optimal ROC margin = 0.0278
Optimal classification threshold (with fairness constraints) = 0.2500
Optimal ROC margin = 0.0278
Optimal classification threshold (with fairness constraints) = 0.2100
Optimal ROC margin = 0.0233
Optimal classification threshold (with fairness constraints) = 0.2500
Optimal ROC margin = 0.0278
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211


The logs above show, for each fold, the selected decision threshold and ROC margin needed to satisfy our fairness constraint. 

Lower margins indicate less aggressive adjustments.

Let’s aggregate results across our folds.  In the table below:

- **DI** (Disparate Impact): ratio of favourable outcomes  

- **F1** (macro/micro/weighted): classification quality  

- **TV distance**: remaining distribution gap on the repaired features  

- **method**: which post-processor was used

In [7]:
report

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.475461,0.68644,0.817751,0.792063,0.183889,origin
1,0.475461,0.68644,0.817751,0.792063,0.183804,unconstrained
2,0.523036,0.686234,0.811561,0.789457,0.000068,barycentre
3,1.033796,0.620186,0.690995,0.705551,0.025841,partial_0.001
4,1.037732,0.718886,0.776899,0.78536,0.183889,ROC
5,0.476313,0.693666,0.822165,0.797155,0.188048,origin
6,0.476313,0.693666,0.822165,0.797155,0.187894,unconstrained
7,0.505581,0.692132,0.81689,0.794358,0.000037,barycentre
8,1.034528,0.614992,0.69148,0.704503,0.025483,partial_0.001
9,0.99929,0.720119,0.780074,0.787688,0.188048,ROC


## 4  Compute average feature importance

Finally, we will revisit our random-forest baseline to see which features drive the income prediction the most.

This helps contextualize which attributes the model relies on - and which we may want to protect.

In [8]:
importance=[]
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    importance.append(list(clf.feature_importances_))

In [9]:
importance=np.array(importance)
print("features", var_list)
print("mean importances", importance.mean(axis=0))

features ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num']
mean importances [0.09313296 0.2046915  0.33826656 0.04915601 0.31475297]


## 5 Conclusions

Our **baseline** (origin/unconstrained) shows the best accuracy but also the biggest gap between protected groups. 

The **barycentre** method virtually erases that gap - but does so by flipping a lot of predictions, which could feel jarring in practice. 

With **partial repair**, you get a handy dial: small tweaks nudge toward parity with minimal impact, while larger tweaks tighten fairness at a greater cost. 

And **ROC post-processing** strikes a nice compromise, cutting disparity quite a bit while keeping f-scores close to where we started.

Looking at feature importance reminds us what the model "cares about" most: **capital gain** and **education level** top the list, with **age** not far behind. If you’re worried about proxying sensitive traits, these are the variables to think hard about - either by guarding them or by designing even earlier interventions.